# Who Owns the Large Buildings in Seattle?

## Problem

The GHGE dataset does not include buildings' owners. Scraping the eRealProperty website for building owners has two limitations:

1. The data quality is poor and many buildings don't have an owner listed.
1. Many corporations with multiple properties set up a separate LLC for each building. There is no straightforward way to trace a child corporation to its parent coroporation. This obfuscates the portfolio size of each company.

We will use the Corporations and Charities Filings System from the Secretary of State to figure this out. The basic process is:
    
1. Start with a company name.
1. Find that company's official name in CCFS.
1. Collect the principals/governors names from that company.
1. Collect all the businesses with those same governors. 
1. Human review to check which companies are connected based on number of overlapping governor, ID number, address, name, etc.
1. Profit?

The CCFS does not have a public API. API endpoints are in the utility methods found in each step. 

Our most up-to-date list of buildings owners, with owner names normalized (e.g., "City of Seattle" and "Seattle City" are both normalized to "City of Seattle"), can be found [here](https://github.com/linnealovespie/BPS/tree/dig_into_owners/experiments/worst_offenders#:~:text=updated_owners_2_15_23.csv). 

In [2]:
import pandas as pd
import numpy as np
import requests
from fuzzywuzzy import fuzz
import json
import os
import re
import geopandas as gp
import util

c:\Users\linne\miniconda3\envs\bps\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Step 0: Isolate to D1 Landlords
1. Get all buildings in d1
2. Look up their tax parcel ID
3. Map that tax parcel ID to a landlord 
4. Aggregate landlords in d1

In [3]:
df_districts = gp.read_file("../data/Council_Districts.geojson")
df = pd.read_csv('../data/2020_Building_Energy_Benchmarking.csv')
df = gp.GeoDataFrame(df, geometry=gp.points_from_xy(df.Longitude, df.Latitude))
util.clean_districts(df, df_districts)

Building WATERWORKS OFFICE & MARINA 2353/ 4088803975 doesn't have a district POINT (-122.33895 47.63575) 
	 Found district 7 for WATERWORKS OFFICE & MARINA
Building NAUTICAL LANDING 2381/ 4088804350 doesn't have a district POINT (-122.34219 47.64306) 
	 Found district 7 for NAUTICAL LANDING
Building UNION HARBOR CONDOMINIUM 2540/ 8807200000 doesn't have a district POINT (-122.33003 47.6401) 
	 Found district 4 for UNION HARBOR CONDOMINIUM
Building THE PIER AT LESCHI 2997/ 6780900000 doesn't have a district POINT (-122.28563 47.59926) 
	 Found district 3 for THE PIER AT LESCHI
Building THE LAKESHORE 3046/ 1180001715 doesn't have a district POINT EMPTY 
Building EDUCARE 3218/ 2895800030 doesn't have a district POINT EMPTY 


In [49]:
df1=df[df["CouncilDistrictCode"]==1]

In [46]:
parcels = pd.read_csv("../data/final_parcels.csv")
parcels.head()
d = pd.Series(parcels.Owner.values, index=parcels.TaxParcelIdentificationNumber).to_dict()
# Map tax ids to landlord name
df1['Landlord'] = df1['TaxParcelIdentificationNumber'].map(lambda row: d.get(row, ""))

In [76]:
landlords=df1.groupby("Landlord")[['TotalGHGEmissions','PropertyGFATotal']]
landlords.size()
df1.value_counts(subset=["Landlord"])

Landlord                            
                                        41
SEATTLE CITY OF                          9
HARSCH INVESTMENT PROPERTIES II LLC      5
MEPT WESTWOOD VILLAGE LLC                4
HARSCH INVESTMENT PROPERTIES LLC         3
                                        ..
ERC SANTOS LLC                           1
EQR-SPRINGLINE LLC                       1
EECKHOUDT JAMES M                        1
DUFFY INVESTMENTS L L C                  1
YSD LLC                                  1
Length: 147, dtype: int64

In [99]:
df1_landlords = landlords.sum()
df1_landlords["NumUnits"] = landlords.size()
# df1_landlords=df1_landlords.drop("")

In [101]:
df1_landlords.to_csv("../data/d1_landlords.csv")

CRE WINSTON LLC         | 	CRE WINSTON, LLC
TESHOME FAMILY LLC      | 	TESHOME FAMILY LLC
TRIAD PIER 70 L L C	    |   TRIAD PIER 70 L.L.C.
BOREN DEVELOPMENT LLC   | 	BOREN DEVELOPMENT, LIMITED LIABILITY COMPANY

In [40]:
result = {
    "BusinessName": "TRIAD PIER 70 L L C", 
    "SearchEntityName": "TRIAD PIER 70 L.L.C."
}

p = re.compile("L[\s.]?L[\s,.]?[PC]" ,flags=re.IGNORECASE)  
p.match("LLC")

result['BusinessName']=result["BusinessName"].replace(",", "")
result['BusinessName']= re.sub(p, "LLC", result['BusinessName'])
result['BusinessName']=result["BusinessName"].replace("LIMITED LIABILITY COMPANY", "LLC") 
result['BusinessName']=result["BusinessName"].replace("LIMITED PARTNERSHIP", "LLC") 
# TODO: keep original business name and add a column that's standardizedname


# Do the same for the search term, so that we have more exact matches
result["SearchEntityName"]=result["SearchEntityName"].replace(",", "").replace(".", "") 
# result["SearchEntityName"]=result["SearchEntityName"].replace("LLP", "LLC") 
result['SearchEntityName']=re.sub(p, "LLC", result['SearchEntityName'])
result["SearchEntityName"]=result["SearchEntityName"].replace("LIMITED LIABILITY COMPANY", "LLC") 
result["SearchEntityName"]=result["SearchEntityName"].replace("LIMITED PARTNERSHIP", "LLC") 

In [2]:
# Utils for finding principals

search_for_business_url = 'https://cfda.sos.wa.gov/api/BusinessSearch/GetBusinessSearchList'

def get_business_search_payload(business_name):
    return {
        'Type': 'BusinessName',
        'SearchType': 'BusinessName',
        'SearchEntityName': business_name,
        'SortType': 'ASC',
        'SortBy': 'Entity Name',
        'SearchValue': business_name,
        'SearchCriteria': 'Contains',
        'IsSearch': 'true',
        'PageID': 1,
        'PageCount': 25,
    }

def get_business_search_results(business_name):
    r = requests.post(search_for_business_url, get_business_search_payload(business_name))
    return json.loads(r.text)

# given one JSON element `result` in the list of search results, standardize
# the business name and address to collapse results into one 
def standardize_result(result):
    # Don't care about the result if it doesn't have an "active" status
    if(result["Status"] != "ACTIVE"): return

    # LLC, LLP, L L C, L.L.C., L.L.C. L.L.P., L.L.P
    # Limited Partnership, Limited liability company
    # Comma before any of the above
    # Just map all the results to be standardized to this name, then drop duplicates based on name? 
    p = re.compile("L[\s.]?L[\s,.]?[PC]" ,flags=re.IGNORECASE)

    result['BusinessName']=result["BusinessName"].replace(",", "")
    result['BusinessName']= re.sub(p, "LLC", result['BusinessName'])
    result['BusinessName']=result["BusinessName"].replace("LIMITED LIABILITY COMPANY", "LLC") 
    result['BusinessName']=result["BusinessName"].replace("LIMITED PARTNERSHIP", "LLC") 

    # Do the same for the search term, so that we have more exact matches
    # TODO: need to add this as an element of the dict
    result["SearchEntityName"]=result["SearchEntityName"].replace(",", "")
    result['SearchEntityName']=re.sub(p, "LLC", result['SearchEntityName'])
    result["SearchEntityName"]=result["SearchEntityName"].replace("LIMITED LIABILITY COMPANY", "LLC") 
    result["SearchEntityName"]=result["SearchEntityName"].replace("LIMITED PARTNERSHIP", "LLC") 
    # Strip addressses of all commas
    result['Address'].replace(",", "")
    return result

def extract_search_results(search_term, search_req_response):
    # TODO: add all the columns, or at least filing status 
    # TODO: collapse all listings with L.L.P or L.L.C or LLC LLP
    res_list = [standardize_result(res) for res in search_req_response]
    # res_list = [[search_term, res['BusinessName'], res['UBINumber'], res['BusinessID'], res['PrincipalOffice']['PrincipalStreetAddress']['FullAddress']] for res in search_req_response]
    res_df = pd.DataFrame(res_list, columns=['SearchTerm', 'BusinessName', 'UBINumber', 'BusinessId', 'Address'])
    # TODO: If there's an exact match, keep only that business 
    # Basically keep a list of exact matches, and build a list of potential matches that we give to human verifiers
    exact_match = res_df.index[res_df['BusinessName'] == search_term].tolist()
    if exact_match:
        res_df = pd.concat([res_df.iloc[[exact_match[0]],:], res_df.drop(exact_match[0], axis=0)], axis=0)
    return res_df
    

# Mark row as potential match: UBI number is a duplicate, or Address is the same
# df.duplicated just sees if that address is already in the dataframe, NOT that the serach term
# and result have the same address. Could add search terms as a subset for duplicated call
def determine_search_matches(search_results_df):
    search_results_df['address_match'] = search_results_df.duplicated(subset=['Address'], keep=False) 
    search_results_df['ubi_match'] = search_results_df.duplicated(subset=['UBINumber'], keep=False)
    search_results_df['id_match'] = search_results_df.duplicated(subset=['BusinessId'], keep=False)

def get_business_details(business_id):
    url = 'https://cfda.sos.wa.gov/api/BusinessSearch/BusinessInformation?businessID={business_id}'.format(business_id=business_id)
    r = requests.get(url)
    return json.loads(r.text)

In [3]:
buildings_and_landlords_df = pd.read_csv('../../experiments/worst_offenders/landlords_with_total_energy_use_2_16_23.csv')

In [4]:
buildings_and_landlords_df.head()

,Unnamed: 0,Owner,BuildingsOwned,TotalSquareFootage,TotalGHGEmissions__metric_tons_,TotalElectricity_kBtu_,TotalSteamUse_kBtu,TotalNaturalGas_kBtu,TotalOtherFuelUse_kBtu,AverageGHGEmissionsIntensity,AverageSiteEUI_kBtu_sf,AverageSourceEUI_kBtu_sf,AverageENERGYSTARScore
0,0,CITY OF SEATTLE,175,17038324,16357,501287744,6826192,247129120,0,1.172674,49.970175,108.376608,72.747899
1,1,UNIVERSITY OF WASHINGTON,43,18445870,39712,679730875,366595771,106707401,0,4.756098,147.265000,321.932500,61.818182
2,2,STATE OF WASHINGTON,17,2768746,3614,88426130,0,59208493,0,2.112500,71.162500,134.625000,55.375000
3,3,KING COUNTY,16,4355059,22535,255701627,16515088,372904431,0,1.750000,66.943750,137.293750,68.083333
4,4,OVERLOOK MAGNOLIA APARTMENTS LLC,11,388815,52,9911319,0,0,0,0.127273,25.418182,71.145455,74.545455


In [5]:
owner_search_list = buildings_and_landlords_df['Owner'].unique()

owner_search_list[:5]

array(['CITY OF SEATTLE', 'UNIVERSITY OF WASHINGTON',
       'STATE OF WASHINGTON', 'KING COUNTY',
       'OVERLOOK MAGNOLIA APARTMENTS LLC'], dtype=object)

In [6]:
n = 200
owner_search_chunks = [owner_search_list[i * n:(i + 1) * n] for i in range((len(owner_search_list) + n - 1) // n )]

In [13]:
owner_search_chunks[0][:10]

array(['CITY OF SEATTLE', 'UNIVERSITY OF WASHINGTON',
       'STATE OF WASHINGTON', 'KING COUNTY',
       'OVERLOOK MAGNOLIA APARTMENTS LLC',
       'CATHOLIC ARCHDIOCESE OF SEATTLE', 'CSHV NWCP SEATTLE LLC',
       'QWEST CORPORATION', 'PUBLIC STORAGE',
       'SELIG HOLDINGS COMPANY LLC'], dtype=object)

In [7]:
def get_potential_company_name_matches(owner_name):
    all_search_results = get_business_search_results(owner_name)
    extracted_results = extract_search_results(owner_name, all_search_results)
    determine_search_matches(extracted_results)
    return extracted_results

In [8]:
get_potential_company_name_matches('MURPHY BALLARD LLC')

,SearchTerm,BusinessName,UBINumber,BusinessId,Address,address_match,ubi_match,id_match
0,MURPHY BALLARD LLC,"MURPHY BALLARD, L.L.C.",602 509 455,860889,"1901 NW MARKET ST, SEATTLE, WA, 98107, UNITED ...",False,False,False


In [9]:
def get_company_list_name_matches(owner_list):
    matches = pd.DataFrame([], columns = ['SearchTerm', 'BusinessName', 'UBINumber', 'BusinessId', 'Address', 'IsMatch'])
    
    for owner in owner_list:
        matches = pd.concat([get_potential_company_name_matches(owner), matches], ignore_index=True)
    
    return matches
    

In [10]:
test_owners = ['BALLARD ASSOC LLC', 'MJWKING LLC', 'MARKET HOLDINGS COMPANY LLC', 'CAR WASH ENTERPRISES INC']

get_company_list_name_matches(test_owners)

C:\Users\linne\AppData\Local\Temp\ipykernel_23000\2009127623.py:5: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  matches = pd.concat([get_potential_company_name_matches(owner), matches], ignore_index=True)
C:\Users\linne\AppData\Local\Temp\ipykernel_23000\2009127623.py:5: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  matches = pd.concat([get_potential_company_name_matches(owner), matches], ignore_index=True)
C:\Users\linne\AppData\Local\Temp\ipykernel_23000\2009127623.py:5: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  matches = pd.concat([get_potential_company_name_matches(owner), matches], ignore_index=T

,SearchTerm,BusinessName,UBINumber,BusinessId,Address,address_match,ubi_match,id_match,IsMatch
0,CAR WASH ENTERPRISES INC,"CAR WASH ENTERPRISES, INC.",578 073 701,603606,"3977 LEARY WAY NW, SEATTLE, WA, 98107-5041, UN...",False,False,False,NaN
1,MARKET HOLDINGS COMPANY LLC,MARKET HOLDINGS COMPANY LLC,603 371 173,829753,"1000 2ND AVE STE 1800, SEATTLE, WA, 98104-3619...",False,False,False,NaN
2,MARKET HOLDINGS COMPANY LLC,"BATS GLOBAL MARKETS HOLDINGS, INC.",604 953 022,1584354,,True,True,True,NaN
3,MARKET HOLDINGS COMPANY LLC,CBOE SERVICES COMPANY,604 953 022,1584354,"8050 MARSHALL DR STE 120, LENEXA, KS, 66214-15...",False,True,True,NaN
4,MARKET HOLDINGS COMPANY LLC,"EVERGREEN MARKET GROUP HOLDINGS, LLC",604 515 369,1318882,"4242 E VALLEY RD, RENTON, WA, 98057-4903, UNIT...",False,False,False,NaN
5,MARKET HOLDINGS COMPANY LLC,EVERGREEN MARKET PROPERTY HOLDINGS LLC,604 513 076,1317864,"4242 E VALLEY RD, RENTON, CA, 98057-4903, UNIT...",False,False,False,NaN
6,MARKET HOLDINGS COMPANY LLC,"FM MARKET HOLDINGS, LLC",602 751 027,699651,"111 N POST ST STE 300, SPOKANE, WA, 99201-4911...",False,False,False,NaN
7,MARKET HOLDINGS COMPANY LLC,"GLOBAL MARKET INSITE INTERNATIONAL HOLDING, INC.",602 498 163,725208,"1100 112TH AVE NE #200, BELLEVUE, WA, 98004, U...",False,False,False,NaN
8,MARKET HOLDINGS COMPANY LLC,"INFORMA MARKETS HOLDINGS, INC.",602 280 823,1070757,"1983 MARCUS AVE STE 250, NEW HYDE PARK, NY, 11...",False,False,False,NaN
9,MARKET HOLDINGS COMPANY LLC,"INLAND NORTHWEST MARKET HOLDINGS, LLC",603 092 077,770143,"505 W RIVERSIDE AVE STE 500, SPOKANE, WA, 9920...",False,False,False,NaN


In [11]:
# Trying this with our first 200 

owner_search_chunk_1 = get_company_list_name_matches(owner_search_chunks[0])

owner_search_chunk_1.head()

C:\Users\linne\AppData\Local\Temp\ipykernel_23000\2009127623.py:5: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  matches = pd.concat([get_potential_company_name_matches(owner), matches], ignore_index=True)
C:\Users\linne\AppData\Local\Temp\ipykernel_23000\2009127623.py:5: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  matches = pd.concat([get_potential_company_name_matches(owner), matches], ignore_index=True)
C:\Users\linne\AppData\Local\Temp\ipykernel_23000\2009127623.py:5: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  matches = pd.concat([get_potential_company_name_matches(owner), matches], ignore_index=T

,SearchTerm,BusinessName,UBINumber,BusinessId,Address,address_match,ubi_match,id_match,IsMatch
0,ASHFORD SEATTLE WATERFRONT L P,ASHFORD SEATTLE WATERFRONT GP LLC,604 268 117,1193505,"14185 DALLAS PKWY STE 1100, DALLAS, TX, 75254-...",True,False,False,NaN
1,ASHFORD SEATTLE WATERFRONT L P,ASHFORD SEATTLE WATERFRONT LEASING LLC,604 417 033,1268461,"14185 DALLAS PKWY STE 1100, DALLAS, TX, 75254-...",True,False,False,NaN
2,ASHFORD SEATTLE WATERFRONT L P,ASHFORD SEATTLE WATERFRONT LP,602 712 212,549855,"14185 DALLAS PKWY STE 1100, DALLAS, TX, 75254,...",False,False,False,NaN
3,ASHFORD SEATTLE WATERFRONT L P,ASHFORD TRS SEATTLE WATERFRONT LLC,604 268 118,1193506,"14185 DALLAS PKWY STE 1100, DALLAS, TX, 75254-...",True,False,False,NaN
4,CRE WINSTON LLC,"CRE WINSTON, LLC",603 154 997,124091,"2930 WESTLAKE AVE N #100, SEATTLE, WA, 98109, ...",False,False,False,NaN


In [12]:
owner_search_chunk_1.to_csv('owner_search_chunk_1.csv')

In [64]:
# 2nd chunk

owner_search_chunk_2 = get_company_list_name_matches(owner_search_chunks[1])

owner_search_chunk_2.head()

owner_search_chunk_2.to_csv('owner_search_chunk_2.csv')

In [66]:
# 3rd chunk
owner_search_chunk_3 = get_company_list_name_matches(owner_search_chunks[2])

owner_search_chunk_3.to_csv('owner_search_chunk_3.csv')

In [67]:
# 4th chunk

owner_search_chunk_4 = get_company_list_name_matches(owner_search_chunks[3])

owner_search_chunk_4.to_csv('owner_search_chunk_4.csv')

In [68]:
# 5th chunk

owner_search_chunk_5 = get_company_list_name_matches(owner_search_chunks[4])

owner_search_chunk_5.to_csv('owner_search_chunk_5.csv')

In [69]:
# 6th chunk

owner_search_chunk_6 = get_company_list_name_matches(owner_search_chunks[5])

owner_search_chunk_6.to_csv('owner_search_chunk_6.csv')

In [70]:
# 7th chunk

owner_search_chunk_7 = get_company_list_name_matches(owner_search_chunks[6])

owner_search_chunk_7.to_csv('owner_search_chunk_7.csv')

In [71]:
# 8th chunk

owner_search_chunk_8 = get_company_list_name_matches(owner_search_chunks[7])

owner_search_chunk_8.to_csv('owner_search_chunk_8.csv')

In [72]:
# 9th chunk

owner_search_chunk_9 = get_company_list_name_matches(owner_search_chunks[8])

owner_search_chunk_9.to_csv('owner_search_chunk_9.csv')

In [73]:
# 10th chunk

owner_search_chunk_10 = get_company_list_name_matches(owner_search_chunks[9])

owner_search_chunk_10.to_csv('owner_search_chunk_10.csv')

In [77]:
# 11th chunk

owner_search_chunk_11 = get_company_list_name_matches(owner_search_chunks[10])

owner_search_chunk_11.to_csv('owner_search_chunk_11.csv')

KeyboardInterrupt: 

In [79]:
all_building_owner_chunks = pd.concat([
    owner_search_chunk_1,
    owner_search_chunk_2,
    owner_search_chunk_3,
    owner_search_chunk_4,
    owner_search_chunk_5,
    owner_search_chunk_6,
    owner_search_chunk_7,
    owner_search_chunk_8,
    owner_search_chunk_9,
    owner_search_chunk_10,
    owner_search_chunk_11
])

In [86]:
all_building_owner_chunks.to_csv('../chunks.csv')

## (Optional) Step 1b

You can ask annotaters to check the possible matches found by the scraping script. 

Proposed categories (hat tip Alice): Match, possible match (will be reviewed by another annotator), not a match

Then you can filter out the "not a match" companies and use the resulting list as input for Step 2.

## Step 2

Fetch the principals for each company found in Step 1.

In [122]:
def get_business_details(business_id):
    url = 'https://cfda.sos.wa.gov/api/BusinessSearch/BusinessInformation?businessID={business_id}'.format(business_id=business_id)
    r = requests.get(url)
    return json.loads(r.text)

def extract_principals(business_res, business_id):
    agent = business_res['Agent']['EntityName']
    rows = [[
        # name of company?
        business_id,
        agent,
        'Entity' if principal['TypeID'] == 'E' else 'Individual',
        principal['PrincipalID'],
         principal['Name'] if principal['TypeID'] == 'E' else principal['FirstName'] + ' ' + principal['LastName']
    ] for principal in business_res['PrincipalsList']]
    return pd.DataFrame(rows, columns=['BusinessId', 'Agent', 'EntityType', 'PrincipalID', 'PrincipalName'])

def get_companies_principals(business_names_df):
    '''
    Takes a DF of companies with BusinessId and returns a DF of each company's principals, 
    with one row for each principal.
    '''
    principals = pd.DataFrame([], columns=['BusinessId', 'Agent', 'EntityType', 'PrincipalID', 'PrincipalName'])
    for business in business_names_df['BusinessId']:
        business_res = get_business_details(business)
        principals = pd.concat([extract_principals(business_res, business), principals], ignore_index=True)
    
    merged_principals = pd.merge(business_names_df, principals, on='BusinessId', how='left')
    
    return merged_principals

In [123]:
#create a table with principals and the business id, then left join on existing table to get business info + principal

principals_search_chunk_1 = get_companies_principals(owner_search_chunk_1)

In [124]:
principals_search_chunk_1.head()

,SearchTerm,BusinessName,UBINumber,BusinessId,Address,address_match,ubi_match,id_match,IsMatch,Agent,EntityType,PrincipalID,PrincipalName
0,ASHFORD SEATTLE WATERFRONT L P,ASHFORD SEATTLE WATERFRONT GP LLC,604 268 117,1193505,"14185 DALLAS PKWY STE 1100, DALLAS, TX, 75254-...",True,False,False,NaN,CORPORATION SERVICE COMPANY,Entity,2826137,ASHFORD CHICAGO SENIOR MEZZ LLC
1,ASHFORD SEATTLE WATERFRONT L P,ASHFORD SEATTLE WATERFRONT GP LLC,604 268 117,1193505,"14185 DALLAS PKWY STE 1100, DALLAS, TX, 75254-...",True,False,False,NaN,CORPORATION SERVICE COMPANY,Individual,3132878,J. ROBINSON HAYS III
2,ASHFORD SEATTLE WATERFRONT L P,ASHFORD SEATTLE WATERFRONT GP LLC,604 268 117,1193505,"14185 DALLAS PKWY STE 1100, DALLAS, TX, 75254-...",True,False,False,NaN,CORPORATION SERVICE COMPANY,Individual,3132879,ROBERT G. HAIMAN
3,ASHFORD SEATTLE WATERFRONT L P,ASHFORD SEATTLE WATERFRONT LEASING LLC,604 417 033,1268461,"14185 DALLAS PKWY STE 1100, DALLAS, TX, 75254-...",True,False,False,NaN,CORPORATION SERVICE COMPANY,Individual,2820892,ROBERT G HAIMAN
4,ASHFORD SEATTLE WATERFRONT L P,ASHFORD SEATTLE WATERFRONT LEASING LLC,604 417 033,1268461,"14185 DALLAS PKWY STE 1100, DALLAS, TX, 75254-...",True,False,False,NaN,CORPORATION SERVICE COMPANY,Individual,3139500,J. ROBINSON HAYS III


In [128]:
# keep getting an error if creating a csv in this directory, so using a redundant file path
principals_search_chunk_1.to_csv('../building_owners/principals/principals_search_chunk_1.csv')

In [129]:
principals_search_chunk_2 = get_companies_principals(owner_search_chunk_2)
principals_search_chunk_2.to_csv('../building_owners/principals/principals_search_chunk_2.csv')

In [130]:
principals_search_chunk_3 = get_companies_principals(owner_search_chunk_3)
principals_search_chunk_3.to_csv('../building_owners/principals/principals_search_chunk_3.csv')

In [131]:
principals_search_chunk_4 = get_companies_principals(owner_search_chunk_4)
principals_search_chunk_4.to_csv('../building_owners/principals/principals_search_chunk_4.csv')

In [132]:
principals_search_chunk_5 = get_companies_principals(owner_search_chunk_5)
principals_search_chunk_5.to_csv('../building_owners/principals/principals_search_chunk_5.csv')

In [133]:
principals_search_chunk_6 = get_companies_principals(owner_search_chunk_6)
principals_search_chunk_6.to_csv('../building_owners/principals/principals_search_chunk_6.csv')

In [134]:
principals_search_chunk_7 = get_companies_principals(owner_search_chunk_7)
principals_search_chunk_7.to_csv('../building_owners/principals/principals_search_chunk_7.csv')

In [135]:
principals_search_chunk_8 = get_companies_principals(owner_search_chunk_8)
principals_search_chunk_8.to_csv('../building_owners/principals/principals_search_chunk_8.csv')

In [136]:
principals_search_chunk_9 = get_companies_principals(owner_search_chunk_9)
principals_search_chunk_9.to_csv('../building_owners/principals/principals_search_chunk_9.csv')

In [137]:
principals_search_chunk_10 = get_companies_principals(owner_search_chunk_10)
principals_search_chunk_10.to_csv('../building_owners/principals/principals_search_chunk_10.csv')

In [138]:
principals_search_chunk_11 = get_companies_principals(owner_search_chunk_11)
principals_search_chunk_11.to_csv('../building_owners/principals/principals_search_chunk_11.csv')

Now we need to find the companies these people are principals for.

Step 1: search for the person using the advanced search

Step 2: go through all the search results, hit the API for each listed business, and look at the principals listed. If one of them matches the person we're looking for, download relevant info. Otherwise, skip.
(NB: the search results don't show the principals' names in either the UI or API response, so you have to look at the full business listing.)


## Step 3

Find every company associated with the governors found in Step 2.

This is slightly convoluted because of the API. The process is:

a) Search for the principal's name using the advanced search API. 

b) Get a paginated list of results. This will *not* include the principal's name because... reasons?

c) For each result, send another request to the business information endpoint to fetch the business details.

d) If the company's principals include the original principal we were looking for, save the business' information.

In [141]:
def get_governor_payload(governor_name, page_num):
    return "Type=Principal&BusinessStatusID=0&SearchEntityName=&SearchType=&BusinessTypeID=0&AgentName=&PrincipalName={governor_name}&StartDateOfIncorporation=&EndDateOfIncorporation=&ExpirationDate=&IsSearch=true&IsShowAdvanceSearch=true&&&AgentAddress%5BIsAddressSame%5D=false&AgentAddress%5BIsValidAddress%5D=false&AgentAddress%5BisUserNonCommercialRegisteredAgent%5D=false&AgentAddress%5BIsInvalidState%5D=false&AgentAddress%5BbaseEntity%5D%5BFilerID%5D=0&AgentAddress%5BbaseEntity%5D%5BUserID%5D=0&AgentAddress%5BbaseEntity%5D%5BCreatedBy%5D=0&&AgentAddress%5BbaseEntity%5D%5BModifiedBy%5D=0&&AgentAddress%5BFullAddress%5D=%2C%20WA%2C%20USA&AgentAddress%5BID%5D=0&&&&AgentAddress%5BState%5D=WA&&AgentAddress%5BCountry%5D=USA&&&&&&&&PrincipalAddress%5BIsAddressSame%5D=false&PrincipalAddress%5BIsValidAddress%5D=false&PrincipalAddress%5BisUserNonCommercialRegisteredAgent%5D=false&PrincipalAddress%5BIsInvalidState%5D=false&PrincipalAddress%5BbaseEntity%5D%5BFilerID%5D=0&PrincipalAddress%5BbaseEntity%5D%5BUserID%5D=0&PrincipalAddress%5BbaseEntity%5D%5BCreatedBy%5D=0&&PrincipalAddress%5BbaseEntity%5D%5BModifiedBy%5D=0&&PrincipalAddress%5BFullAddress%5D=%2C%20WA%2C%20USA&PrincipalAddress%5BID%5D=0&&&&PrincipalAddress%5BState%5D=&&PrincipalAddress%5BCountry%5D=USA&&&&&&IsHostHomeSearch=&IsPublicBenefitNonProfitSearch=&IsCharitableNonProfitSearch=&IsGrossRevenueNonProfitSearch=&IsHasMembersSearch=&IsHasFEINSearch=&NonProfit%5BIsNonProfitEnabled%5D=false&NonProfit%5BchkSearchByIsHostHome%5D=false&NonProfit%5BchkSearchByIsPublicBenefitNonProfit%5D=false&NonProfit%5BchkSearchByIsCharitableNonProfit%5D=false&NonProfit%5BchkSearchByIsGrossRevenueNonProfit%5D=false&NonProfit%5BchkSearchByIsHasMembers%5D=false&NonProfit%5BchkSearchByIsHasFEIN%5D=false&NonProfit%5BFEINNoSearch%5D=&NonProfit%5BchkIsHostHome%5D%5Bnone%5D=false&NonProfit%5BchkIsHostHome%5D%5Byes%5D=false&NonProfit%5BchkIsHostHome%5D%5Bno%5D=false&NonProfit%5BchkIsPublicBenefitNonProfit%5D%5Bnone%5D=false&NonProfit%5BchkIsPublicBenefitNonProfit%5D%5Byes%5D=false&NonProfit%5BchkIsPublicBenefitNonProfit%5D%5Bno%5D=false&NonProfit%5BchkIsCharitableNonProfit%5D%5Bnone%5D=false&NonProfit%5BchkIsCharitableNonProfit%5D%5Byes%5D=false&NonProfit%5BchkIsCharitableNonProfit%5D%5Bno%5D=false&NonProfit%5BchkIsGrossRevenueNonProfit%5D%5Bnone%5D=false&NonProfit%5BchkIsGrossRevenueNonProfit%5D%5Byes%5D=false&NonProfit%5BchkIsGrossRevenueNonProfit%5D%5Bno%5D=false&NonProfit%5BchkIsGrossRevenueNonProfit%5D%5Bover500k%5D=false&NonProfit%5BchkIsGrossRevenueNonProfit%5D%5Bunder500k%5D=false&NonProfit%5BchkIsHasMembers%5D%5Bnone%5D=false&NonProfit%5BchkIsHasMembers%5D%5Byes%5D=false&NonProfit%5BchkIsHasMembers%5D%5Bno%5D=false&NonProfit%5BchkIsHasFEIN%5D%5Byes%5D=false&NonProfit%5BchkIsHasFEIN%5D%5Bno%5D=false&PageID={page_num}&PageCount=100".format(governor_name=governor_name, page_num=page_num)

governor_headers = {
    'Accept': 'application/json, text/plain, */*',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'en-US,en;q=0.9',
'Connection': 'keep-alive',
'Content-Length': '2778',
'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
'DNT': '1',
'Host': 'cfda.sos.wa.gov',
'Origin': 'https://ccfs.sos.wa.gov',
'Referer': 'https://ccfs.sos.wa.gov/',
'Sec-Fetch-Dest': 'empty',
'Sec-Fetch-Mode': 'cors',
'Sec-Fetch-Site': 'same-site',
'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
'sec-ch-ua-mobile': '?0',
'sec-ch-ua-platform': "macOS"
}

def get_governor_results_json(governor_name, page_num):
    r = requests.post('https://cfda.sos.wa.gov/api/BusinessSearch/GetAdvanceBusinessSearchList', data=get_governor_payload(governor_name, page_num), headers=governor_headers)
    return json.loads(r.text)

def get_all_governor_search_results(governor_name):
    n = 1
    res_length = 100
    search_results = []
    
    while res_length == 100:
        res = get_governor_results_json(governor_name, n)
        search_results.append(res)
        n += 1
        res_length = len(res)
    
    return search_results

def get_governors_from_all_results_pages(governor_name):
    search_results = get_all_governor_search_results(governor_name)
    business_ids = [res['BusinessId'] for res in search_reults]
    
    # should include company name, see below
    principals = pd.DataFrame([], columns=['BusinessId', 'Agent', 'EntityType', 'PrincipalID', 'PrincipalName'])
    
    for id in business_ids:
        business_json = get_business_details(business_id)
        principals_df = extract_principals(business_json, id) # we should add the company name--write a variant of extract_principals
        # check if the principal is included
        if len(principals_df[principals_df['PrincipalName'] == governor_name]) > 0:
            pd.concat([extract_principals(business_res, business), principals], ignore_index=True)
    
    return principals_df

In [ ]:
NB: I have not tested these functions, you will have to:
        - add the business name to the results returned in get_governors_from_all_results_pages
        - test the methods to make sure they work

You should be able to run the get_governors_from_all_results_pages for each row of the CSV generated in Step 2 to get a full list of principals.

Then you can have humans review the outcomes.

If you want to find out what a particular person is involved in, you can just do this process starting with step 3.